In [ ]:
using DataFrames

using GZip

using OnePiece

In [ ]:
function _pair(bl)

    ke_va = Dict()

    be = "_table_begin\n"

    if contains(bl, be)

        pa, ta = split(bl, be)

        pa_ = split(pa, '\n')

        ta_ = split(ta, '\n')[1:(end - 1)]

        wh1 = popat!(pa_, length(pa_))

        wh2 = popat!(ta_, length(ta_))

        if "$(wh1)_table_end" != wh2

            error()

        end

        li_ = split.(ta_, '\t')

        da = DataFrame([co => [] for co in popat!(li_, 1)])

        for li in li_

            push!(da, li)

        end

        ke_va["da"] = da

    else

        pa_ = split(bl, '\n')

    end

    for li in split(pa, '\n', keepempty = false)[1:(end - 1)]

        ke, va = split(li, " = ", limit = 2)

        ke = lstrip(ke, '!')

        id = 2

        while haskey(ke_va, ke)

            ke = "$ke.$id"

            id += 1

        end

        ke_va[ke] = va

    end

    ke_va

end

In [ ]:
gs = "GSE122404"#"GSE107011"

di = "/Users/kwat/Downloads"

In [ ]:
na = "$(gs)_family.soft.gz"

pa = joinpath(di, na)

if !ispath(pa)

    download("ftp://ftp.ncbi.nlm.nih.gov/geo/series/$(gs[1:end-3])nnn/$gs/soft/$na", pa)

end

pl_di = Dict()

sa_di = Dict()

for bl in split(read(GZip.open(pa, "r"), String), '^')[2:end]

    he, bl = split(bl, '\n', limit = 2)

    ty = split(he, " = ")[1]

    if !(ty in ["PLATFORM", "SAMPLE"])

        println("Skipping $ty:\n$bl")

        continue

    end

    ke_va = _pair(bl)

    if ty == "PLATFORM"

        pl_di[ke_va["Platform_geo_accession"]] = ke_va

    elseif ty == "SAMPLE"

        sa_di[ke_va["Sample_title"]] = ke_va

    end

end

In [ ]:
sa_da = Dict(sa => pop!(di, "da") for (sa, di) in sa_di)

In [ ]:
fe_x_sa_x_an = DataFrame([sa => collect(values(di)) for (sa, di) in sa_di])

insertcols!(fe_x_sa_x_an, 1, "Feature" => collect(keys(collect(values(sa_di))[1])))

In [ ]:
fe_x_sa_x_nu = DataFrame([
    sa => parse.(Float64, da[:, findfirst(names(da) .== "VALUE")]) for (sa, da) in sa_da
])

insertcols!(fe_x_sa_x_nu, 1, "Feature" => collect(values(sa_da))[1][:, 1])

In [ ]:
fe_ = fe_x_sa_x_nu[!, "Feature"]

In [ ]:
pl = pl_di[sa_di["D458_Resistant_DMSO_1"]["Sample_platform_id"]]["da"]